<a href="https://colab.research.google.com/github/YaminMahdi/All-.ipynb/blob/master/LAB2_Task_by_193_15_1071.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Spark").getOrCreate()
spark

In [ ]:
!wget https://raw.githubusercontent.com/YaminMahdi/All-.ipynb/master/results.csv
!wget https://raw.githubusercontent.com/YaminMahdi/All-.ipynb/master/pokemon.csv
!wget https://raw.githubusercontent.com/YaminMahdi/All-.ipynb/master/Literacy.csv


In [14]:
d1 = spark.read.csv('results.csv', header=True, sep=",", inferSchema=True)
d2 = spark.read.csv('pokemon.csv', header=True, sep=",", inferSchema=True)
d3 = spark.read.csv('Literacy.csv', header=True, sep=",", inferSchema=True)

d1.show(2)
d2.show(2)
d3.show(2)

+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  false|
|1873-03-08|  England| Scotland|         4|         2|  Friendly| London| England|  false|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
only showing top 2 rows

+---+--------------+---------+-----------+------------------------+----------+----------------+------------+-----------+------------+-----------+------+------+--------+---------+----------------+---------+---------+--------------+------------+----+------+-------+---------+----------+-----+----------+---------------+---------------+-----------+---------------+----------+----------+---------------+----------+--------------+----

## Calculate Age Function

In [22]:
from pyspark.sql.functions import lit, to_date, current_date, datediff

def cal_age(date):
  age = datediff(current_date(), to_date(date)) 
  x=  ((age/365))
  return x

In [23]:
d1.select(d1.date, cal_age(d1.date).alias("age (years)")).show(5)

+----------+------------------+
|      date|       age (years)|
+----------+------------------+
|1872-11-30|149.87123287671233|
|1873-03-08| 149.6027397260274|
|1874-03-07| 148.6054794520548|
|1875-03-06|147.60821917808218|
|1876-03-04| 146.6109589041096|
+----------+------------------+
only showing top 5 rows



## DataSet Join

In [21]:
print("cross join")
d3.join(d2, d3._c0 == d2._c0, "cross").select(d2["_c0"],d3["State"],d2["species"]).show(3)

print("outer join")
d3.join(d2, d3._c0 == d2._c0, "outer").select(d2["_c0"],d3["State"],d2["japanese_name"]).show(3)

print("full join")
d3.join(d2, d3._c0 == d2._c0, "full").select(d2["_c0"],d3["District"],d2["german_name"]).show(3)

print("left join")
d3.join(d2, d3._c0 == d2._c0, "left").select(d2["_c0"],d3["Literacy"],d2["ability_hidden"]).show(3)

print("right join")
d3.join(d2, d3._c0 == d2._c0, "right").select(d2["_c0"],d3["Literacy"],d2["defense"]).show(3)


# cross, outer, full, fullouter, full_outer, left, leftouter, left_outer, right, rightouter, right_outer, semi, leftsemi, left_semi, anti, leftanti and left_anti.

cross join
+---+------------+------------+
|_c0|       State|     species|
+---+------------+------------+
|  1| Maharashtra|Seed Pokémon|
|  2| West Bengal|Seed Pokémon|
|  3|   Karnataka|Seed Pokémon|
+---+------------+------------+
only showing top 3 rows

outer join
+---+------------------+----------------------+
|_c0|             State|         japanese_name|
+---+------------------+----------------------+
|148|     Uttar Pradesh|     ガルーラ (Garura)|
|463|           Haryana|   デオキシス (Deoxys)|
|471| Jammu and Kashmir|ポッタイシ (Pottaishi)|
+---+------------------+----------------------+
only showing top 3 rows

full join
+---+-------------+-----------+
|_c0|     District|german_name|
+---+-------------+-----------+
|148|  Maharajganj|    Kangama|
|463| Mahendragarh|     Deoxys|
|471|      Kupwara|    Pliprin|
+---+-------------+-----------+
only showing top 3 rows

left join
+---+--------+--------------+
|_c0|Literacy|ability_hidden|
+---+--------+--------------+
|  1|   84.53|   Chloro

In [24]:
watch_name =spark.createDataFrame([[1, "A"," Rolex",1899,1944],[2, "f"," Rolex",2899,1924],[3, "g"," Rolex",1599,1990],[4, "z"," Rolex",7899,2001],[5, "h"," Rolex",799,1844],[6, "r"," Rolex",1899,1944],[7, "u"," Rolex",2899,1924],[8, "i"," Rolex",1599,1990],[9, "o"," Rolex",7899,2001],[10, "p"," Rolex",799,1844]], ["id", "name","brand","price","established"])
watch_pop =spark.createDataFrame([[1, "95%"],[2, "75%"],[3, "80%"],[4, "90%",],[5, "90%",],[6, "65%"],[7, "75%"],[8, "85%"],[9, "94%",],[10, "50%",]], ["id", "Popularity"])
watch_name.show()
watch_pop.show()
watch_name.join(watch_pop, watch_name.id == watch_pop.id, "inner").select(watch_name["id"],watch_name["name"],watch_pop["Popularity"]).show()
watch_name.join(watch_pop, watch_name.id == watch_pop.id, "outer").select(watch_name["id"],watch_name["name"],watch_pop["Popularity"]).show()
watch_name.join(watch_pop, watch_name.id == watch_pop.id, "cross").select(watch_name["id"],watch_name["name"],watch_pop["Popularity"]).show()
watch_name.join(watch_pop, watch_name.id == watch_pop.id, "full_outer").select(watch_name["id"],watch_name["name"],watch_pop["Popularity"]).show()
watch_name.join(watch_pop, watch_name.id == watch_pop.id, "full").select(watch_name["id"],watch_name["name"],watch_pop["Popularity"]).show()

+---+----+------+-----+-----------+
| id|name| brand|price|established|
+---+----+------+-----+-----------+
|  1|   A| Rolex| 1899|       1944|
|  2|   f| Rolex| 2899|       1924|
|  3|   g| Rolex| 1599|       1990|
|  4|   z| Rolex| 7899|       2001|
|  5|   h| Rolex|  799|       1844|
|  6|   r| Rolex| 1899|       1944|
|  7|   u| Rolex| 2899|       1924|
|  8|   i| Rolex| 1599|       1990|
|  9|   o| Rolex| 7899|       2001|
| 10|   p| Rolex|  799|       1844|
+---+----+------+-----+-----------+

+---+----------+
| id|Popularity|
+---+----------+
|  1|       95%|
|  2|       75%|
|  3|       80%|
|  4|       90%|
|  5|       90%|
|  6|       65%|
|  7|       75%|
|  8|       85%|
|  9|       94%|
| 10|       50%|
+---+----------+

+---+----+----------+
| id|name|Popularity|
+---+----+----------+
|  7|   u|       75%|
|  6|   r|       65%|
|  9|   o|       94%|
|  5|   h|       90%|
|  1|   A|       95%|
| 10|   p|       50%|
|  3|   g|       80%|
|  8|   i|       85%|
|  2|   f|    